In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from tqdm import tqdm
import pickle
import seaborn as sns
import multiprocessing
import re
from importlib import reload

In [2]:
import sys
sys.path.insert(1, '/project/cper_neon_aop/cper_hls_veg_models/biomass/params')
sys.path.insert(1, '/project/cper_neon_aop/hls_nrt/fit')
sys.path.insert(1, '/project/cper_neon_aop/hls_nrt/extract')

In [3]:
#reload(sys.modules["cper_bm_params_fit_year_plot_nobai"])
from cper_bm_params_fit_year_transect import *

In [4]:
#reload(sys.modules["hpc_setup"])
from hpc_setup import launch_dask
from ml_setup import fit_ols, fit_dnn

In [5]:
#import dask
#client = dask.distributed.get_client()
#client.cluster.close()
#client.close()

In [6]:
if backend == 'dask':
    from hpc_setup import launch_dask
    import psutil
    interface_prefs = [
        'ibp175s0',
        'ibp59s0',
        'enp24s0f0',
        'ens7f0',
        'ibp24s0',
        'ib0'
    ]
    interface = [x for x in interface_prefs if x in list(psutil.net_if_addrs().keys())][0]
    if len(interface) == 0:
        print('ERROR: Preferred interfaces not found on node!')
    else:
        print(interface)
    num_jobs=128
    client = launch_dask(cluster_loc=cluster_loc,
                         num_jobs=num_jobs,
                         num_processes=2,
                         num_threads_per_processes=2,
                         mem_gb_per=12.0,
                         partition='atlas', 
                         duration='36:00:00',
                         slurm_opts={'interface': interface},
                         extra_directives=["--account=cper_neon_aop"],
                         wait_proportion=0.25,
                        wait_timeout=240,
                        debug=False)
    display(client)
    nthreads = len(client.nthreads())
else:
    nthreads = multiprocessing.cpu_count()

ib0
   setting up cluster on HPC...


<Client: 'tcp://172.21.15.107:44081' processes=6 threads=12, memory=134.10 GiB>

In [7]:
from distributed import get_client
client = get_client()
nthreads = len(client.nthreads())

In [8]:
display(client)
#nthreads = len(client.nthreads())

<Client: 'tcp://172.21.15.107:44081' processes=6 threads=12, memory=134.10 GiB>

In [9]:
import torch
if torch.has_cuda and use_cuda:
    device='cuda'
    print(torch.cuda.device_count())
else:
    device='cpu'
print(device)

cpu


In [10]:
#reload(sys.modules["ml_setup"])
from ml_setup import make_model_dictionary
from ml_setup import run_ml_models_bootstrap_yr

In [11]:
with open('./results/tmp/ml_train_cper_bm_transect_cv_Year_tuneby_Year_results.pk', 'rb') as f:
    mod_dict = pd.compat.pickle_compat.load(f)

In [12]:
df = load_df(inPATH, ['Date', 'Date_mean'])

In [13]:
#import itertools
#combos = list(itertools.combinations(df['Year'].unique(), 3))

In [14]:
#combos[-1]

In [15]:
for k in mod_dict:
    if k not in ['CPER_2022', 'OLS_2022', 'OLS', 'DNN']:
        param_best = {}
        param_best_dict = {p: [] for p in mod_dict[k]['param_grid'].keys()}
        for y in mod_dict[k]['tune_results'].keys():
            param_best_dict_tmp = mod_dict[k]['tune_results'][y]['params'][mod_dict[k]['tune_results'][y]['mean_test_' + mod_dict[k]['tune_refit']].argmax()]
            for p in param_best_dict_tmp.keys():
                param_best_dict[p].append(param_best_dict_tmp[p])
        #print(param_best_dict)
        for p in param_best_dict:
            if all([type(i) in [float, np.float64] for i in param_best_dict[p]]):
                param_best[p] = np.mean(param_best_dict[p])
            elif all([type(i) in [int, np.int64] for i in param_best_dict[p]]):
                param_best[p] = int(np.mean(param_best_dict[p]))
            elif all([i is None for i in param_best_dict[p]]):
                param_best[p] = None
            else:
                print('ERROR')
        mod_dict[k]['param_best'] = param_best

/project/cper_neon_aop/conda_envs/py_ml_env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/project/cper_neon_aop/conda_envs/py_ml_env/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [16]:
mod_dict['SVR']['base_mod']['SVR'].cache_size = 10000
mod_dict['RF']['base_mod']['RF'].warm_start = False
#mod_dict['GBR']['base_mod']['GBR'].warm_start = False
mod_dict['HGBR']['base_mod']['GBR'].warm_start = False

In [17]:
[k + str(mod_dict[k]['fit']) for k in mod_dict]

['CPER_2022True',
 'OLS_2022True',
 'OLSTrue',
 'LASSOTrue',
 'PLSTrue',
 'PCRTrue',
 'SVRTrue',
 'RFTrue',
 'GBRFalse',
 'HGBRTrue',
 'MLPFalse',
 'DNNFalse']

In [18]:
mod_dict['DNN']['fit'] = False
mod_dict['CPER_2022']['fit'] = False
mod_dict['GBR']['fit'] = False

In [19]:
if 'BAI_236' not in var_names or 'NDII7' not in var_names or 'NIR1' not in var_names:
    mod_dict['OLS_2022']['fit'] = False

In [20]:
if 'transect' in inPATH:
    df['Plot'] = df[id_col].transform(lambda x: '_'.join(x.split('_')[:-1]))
    agg_plot=True
else:
    agg_plot=False

In [21]:
#client.restart()
#import distributed
#client = [x for x in distributed.client._global_clients.values()][0]
#display(client)
#nthreads = len(client.nthreads())

In [22]:
#reload(sys.modules["ml_setup"])
#from ml_setup import run_ml_models_bootstrap_yr

In [23]:
df_out = run_ml_models_bootstrap_yr(nickname, mod_dict, df, y_col, date_col, var_names,
                               tuneby_group,
                               backend, nthreads,
                               cper_mod_xfrm, cper_mod_xfrm_func, client,
                               retune_bootstrap=True,
                                    agg_plot=agg_plot,
                                   save_path='./results/tmp/vor_bootstrap_by_year_transect_v2.csv')

Running 3-year combos
120
All iterations run for all models. Skipping combos.
Running 4-year combos
210
All iterations run for all models. Skipping combos.
Running 5-year combos
252
All iterations run for all models. Skipping combos.
Running 6-year combos
210
All iterations run for all models. Skipping combos.
Running 7-year combos
120
All iterations run for all models. Skipping combos.
Running 8-year combos
45
All iterations run for all models. Skipping combos.
Running 9-year combos
10
All iterations run for all models. Skipping combos.
Running 10-year combos
1
All iterations run for all models. Skipping combos.


In [24]:
#df_out.to_csv('./results/tmp/vor_bootstrap_by_year_test.csv')

In [25]:
client.cluster.close()
client.close()